1. 將物件中所有的面都畫出來

In [ ]:
import os
import time
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.TopAbs import TopAbs_FACE

display, start_display, add_menu, add_function_to_menu = init_display()

file_path = r"C:\NTHU\IRTI-Project\my_box.igs"

# 影像的保存路徑
image_directory = r"C:\NTHU\IRTI-Project\Images"
if not os.path.exists(image_directory):
    os.makedirs(image_directory)


reader = IGESControl_Reader()
status = reader.ReadFile(file_path)
if status == 1:  
    reader.TransferRoots()
    shape = reader.Shape()

    index = 1
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())

        display.EraseAll()

        display.DisplayShape(face, update=True, color='BLUE', transparency=0.5)  # 可以調整顏色和透明度
        display.FitAll()

        time.sleep(2)  # 暫停兩秒

        # 儲存每個面的影像
        image_path = os.path.join(image_directory, f'face_{index}.png')
        display.View.Dump(image_path)
        print(f'Saved image of face {index} to {image_path}')

        # 移動到下一個面
        index += 1
        explorer.Next()
else:
    print("Failed to read the file.")
start_display()

2. 去計算每個面的一些資訊(面積，上下界，面的類型)

In [ ]:
import os
import time
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepTools import breptools_UVBounds
from OCC.Core.GeomLProp import GeomLProp_SLProps
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GProp import GProp_GProps
from OCC.Core.gp import gp_Pnt

file_path = r"C:\NTHU\IRTI-Project\my_box.igs"

reader = IGESControl_Reader()
status = reader.ReadFile(file_path)
if status == 1: 
    reader.TransferRoots()
    shape = reader.Shape()

    index = 1
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())
        
        adaptor_surface = BRepAdaptor_Surface(face, True)
        surface_type = adaptor_surface.GetType()              # 獲取面的類型
    
        # 獲取面的邊界
        u_min, u_max, v_min, v_max = adaptor_surface.FirstUParameter(), adaptor_surface.LastUParameter(), adaptor_surface.FirstVParameter(), adaptor_surface.LastVParameter()
    
        # 計算面積
        gprops = GProp_GProps()                             # 用於獲取面的質量等信息
        brepgprop_SurfaceProperties(face, gprops)           # 計算面的面積
        area = gprops.Mass()                                # 獲取面的面積

        print(f"Face {index}:")
        print(f"  Type: {surface_type}")
        print(f"  Bounds in U: {u_min} to {u_max}")
        print(f"  Bounds in V: {v_min} to {v_max}")
        print(f"  Area: {area}")

        index += 1
        explorer.Next()

3. 獲取每個面的邊線

In [ ]:
import os
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face, topods_Edge
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface, BRepAdaptor_Curve
from OCC.Core.BRepTools import breptools_UVBounds
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_LinearProperties
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge
from OCC.Core.GeomAPI import GeomAPI_PointsToBSpline

# IGS文件路径
file_path = r"C:\NTHU\IRTI-Project\my_box.igs"

reader = IGESControl_Reader()
status = reader.ReadFile(file_path)
if status == 1:
    reader.TransferRoots()
    shape = reader.Shape()

    index = 1
    face_explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while face_explorer.More():
        face = topods_Face(face_explorer.Current())
        print(f"Face {index}:")

        edge_explorer = TopExp_Explorer(face, TopAbs_EDGE)
        edge_index = 1
        while edge_explorer.More():
            edge = topods_Edge(edge_explorer.Current())
            
            # 获取边的长度
            gprops_edge = GProp_GProps()
            brepgprop_LinearProperties(edge, gprops_edge)
            length = gprops_edge.Mass()  # 边的长度
            
            # 创建边的适配器来进一步获取信息
            curve_adaptor = BRepAdaptor_Curve(edge)
            curve_type = curve_adaptor.GetType()
            
            # 输出边信息
            print(f"  Edge {edge_index}: Length = {length}, Type = {curve_type}")
            
            edge_index += 1
            edge_explorer.Next()

        index += 1
        face_explorer.Next()
else:
    print("Failed to read the file.")

4. 自動尋找曲面上的一點，並且計算其法向量

In [7]:
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.BRepTools import breptools_UVBounds
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.GeomAbs import GeomAbs_Cylinder

# 读取IGES文件
file_path = r"C:\NTHU\IRTI-Project\my_cylinder.igs"
reader = IGESControl_Reader()
status = reader.ReadFile(file_path)
if status == 1:  # 成功读取文件
    reader.TransferRoots()
    shape = reader.Shape()

    # 遍历所有的面
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())
        adaptor_surface = BRepAdaptor_Surface(face, True)
        
        # 检查面的类型是否为圆柱面
        if adaptor_surface.GetType() == GeomAbs_Cylinder:
            # 获取参数范围
            u_min, u_max, v_min, v_max = breptools_UVBounds(face)

            # 选择圆柱面上的一点，这里我们选择中点
            u = (u_min + u_max) / 2
            v = (v_min + v_max) / 2
            props = GeomLProp_SLProps(adaptor_surface.Surface(), u, v, 1, 0.01)
            if props.IsNormalDefined():
                normal = props.Normal()
                # 获取对应的3D点坐标
                pnt = adaptor_surface.Value(u, v)
            
            print(f"Point on cylindrical surface: ({pnt.X()}, {pnt.Y()}, {pnt.Z()})")
            print(f"normal vector: ({normal})")
            # 只处理第一个圆柱面
            break

        # 继续寻找下一个面
        explorer.Next()
else:
    print("Failed to read the file.")

C:\Users\nick2\AppData\Local\Temp\ipykernel_7628\2913289348.py:20: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())
C:\Users\nick2\AppData\Local\Temp\ipykernel_7628\2913289348.py:20: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())
C:\Users\nick2\AppData\Local\Temp\ipykernel_7628\2913289348.py:20: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())
